<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#!/usr/bin/env python3
# ═══════════════════════════════════════════════════════════════════════════
# MY-AI-GIZMO AGGRESSIVE URL CAPTURE - GUARANTEED LINK
# ═══════════════════════════════════════════════════════════════════════════

import os
import sys
import subprocess
import shutil
import time
import re
import threading
from pathlib import Path

print("🚀 MY-AI-GIZMO - GUARANTEED GRADIO LINK")
print("="*70)

# ═══════════════════════════════════════════════════════════════════════════
# HELPERS
# ═══════════════════════════════════════════════════════════════════════════

def run_cmd(cmd, check=False, quiet=True, timeout=None):
    try:
        if quiet:
            return subprocess.run(cmd, check=check, capture_output=True, text=True, timeout=timeout)
        else:
            return subprocess.run(cmd, check=check, timeout=timeout)
    except:
        return None

def safe_pip(packages):
    if isinstance(packages, str):
        packages = [packages]
    for pkg in packages:
        try:
            subprocess.run([sys.executable, "-m", "pip", "install", "-q", pkg],
                         check=False, timeout=300, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        except:
            pass

def extract_urls(text):
    """Extract both local and public URLs"""
    urls = {}

    # Pattern for Gradio public URLs
    public_match = re.search(r'https://[a-z0-9]+\.gradio\.live', text, re.IGNORECASE)
    if public_match:
        urls['public'] = public_match.group(0)

    # Pattern for local URLs
    local_patterns = [
        r'http://127\.0\.0\.1:\d+',
        r'http://localhost:\d+',
        r'Running on local URL:\s+(http://[^\s]+)',
        r'Running on public URL:\s+(https://[^\s]+)'
    ]

    for pattern in local_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            if 'local URL' in pattern or '127.0.0.1' in pattern or 'localhost' in pattern:
                urls['local'] = match.group(1) if match.lastindex else match.group(0)
            elif 'public URL' in pattern:
                urls['public'] = match.group(1) if match.lastindex else match.group(0)

    return urls

def print_url_box(local_url, public_url, hw):
    box = "="*70
    print("\n" + box)
    print("🎉"*35)
    print(box)
    print("✅ SERVER IS RUNNING!")
    print(box)

    if public_url:
        print(f"\n🌐 PUBLIC URL (SHARE THIS):\n")
        print(f"   {public_url}\n")

    if local_url:
        print(f"🏠 LOCAL URL:\n")
        print(f"   {local_url}\n")

    print(box)
    print(f"💾 All chats save to Google Drive")
    print(f"🖥️  Hardware: {hw}")
    print(f"⏹️  Press Ctrl+C to stop")
    print(box + "\n")

# ═══════════════════════════════════════════════════════════════════════════
# PATHS
# ═══════════════════════════════════════════════════════════════════════════

DRIVE_BASE = Path("/content/drive/MyDrive/MY-AI-Gizmo")
REPO_DIR = DRIVE_BASE / "MY-AI-Gizmo-working"
MODELS_DIR = DRIVE_BASE / "models"
USER_DATA_DIR = DRIVE_BASE / "user_data"
MODEL_FILE = MODELS_DIR / "llama-2-7b.Q4_K_M.gguf"

# ═══════════════════════════════════════════════════════════════════════════
# STEP 1: MOUNT DRIVE
# ═══════════════════════════════════════════════════════════════════════════

print("\n📂 Step 1: Google Drive...")

try:
    from google.colab import drive
    mydrive = Path("/content/drive/MyDrive")

    if mydrive.exists() and mydrive.is_dir():
        print("   ✅ Mounted")
    else:
        drive.mount('/content/drive', force_remount=True)
        print("   ✅ Mounted")

    if not mydrive.exists():
        raise Exception("Mount failed")

except Exception as e:
    print(f"   ❌ Error: {e}")
    raise SystemExit(1)

for d in [DRIVE_BASE, MODELS_DIR, USER_DATA_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# ═══════════════════════════════════════════════════════════════════════════
# STEP 2-3: SYSTEM + REPO
# ═══════════════════════════════════════════════════════════════════════════

print("\n📦 Step 2: System...")
run_cmd(["apt-get", "update", "-qq"])
run_cmd(["apt-get", "install", "-y", "-qq", "build-essential", "cmake", "git", "wget"])
print("   ✅ Done")

print("\n📥 Step 3: Repository...")

if (REPO_DIR / "server.py").exists():
    print("   ✅ Exists")
else:
    REPO_DIR.parent.mkdir(parents=True, exist_ok=True)
    os.chdir(REPO_DIR.parent)
    run_cmd(["git", "clone", "https://github.com/gitleon8301/MY-AI-Gizmo-working.git"])
    print("   ✅ Cloned")

os.chdir(REPO_DIR)

# ═══════════════════════════════════════════════════════════════════════════
# STEP 4: PACKAGES
# ═══════════════════════════════════════════════════════════════════════════

print("\n🐍 Step 4: Packages (quick install)...")

safe_pip(["setuptools", "wheel", "numpy", "requests", "tqdm", "pyyaml"])

env = dict(os.environ)
env["CMAKE_ARGS"] = "-DLLAMA_CUBLAS=off -DLLAMA_BUILD_SERVER=ON"
subprocess.run([sys.executable, "-m", "pip", "install", "--force-reinstall", "--no-cache-dir", "-q",
               "llama-cpp-python[server]"],
              check=False, env=env, timeout=600, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

safe_pip(["torch", "transformers", "gradio>=3.50.0", "accelerate", "markdown",
         "Pillow", "safetensors", "sentencepiece", "protobuf", "flask_cloudflared"])

print("   ✅ Done")

# ═══════════════════════════════════════════════════════════════════════════
# STEP 5: FIND LLAMA-SERVER
# ═══════════════════════════════════════════════════════════════════════════

print("\n🔍 Step 5: Finding llama-server...")

llama_server_path = None
check_paths = [
    Path(sys.executable).parent / "llama-server",
    Path(sys.prefix) / "bin" / "llama-server",
]

try:
    import llama_cpp
    llama_cpp_dir = Path(llama_cpp.__file__).parent
    check_paths.insert(0, llama_cpp_dir / "server" / "llama-server")
except:
    pass

for path in check_paths:
    if path.exists():
        llama_server_path = str(path)
        print(f"   ✅ Found: {llama_server_path}")
        break

if not llama_server_path:
    result = run_cmd(['which', 'llama-server'])
    if result and result.returncode == 0 and result.stdout.strip():
        llama_server_path = result.stdout.strip()
        print(f"   ✅ Found: {llama_server_path}")
    else:
        llama_server_path = "llama-server"
        print(f"   ⚠️  Using default: {llama_server_path}")

# ═══════════════════════════════════════════════════════════════════════════
# STEP 6-7: FIX UI AND LLAMA
# ═══════════════════════════════════════════════════════════════════════════

print("\n🔧 Step 6: Fixing UI...")

ui_file = REPO_DIR / "modules" / "ui.py"
if ui_file.exists():
    with open(ui_file, 'r', encoding='utf-8') as f:
        content = f.read()

    if '# COLAB_FIX_APPLIED' not in content:
        lines = content.split('\n')
        fixed_lines = []
        in_theme_block = False
        theme_block_indent = 0

        for line in lines:
            if 'if not shared.args.old_colors:' in line and not in_theme_block:
                fixed_lines.append('    # COLAB_FIX_APPLIED')
                fixed_lines.append('    # ' + line.strip())
                in_theme_block = True
                theme_block_indent = len(line) - len(line.lstrip())
                continue

            if in_theme_block:
                current_indent = len(line) - len(line.lstrip()) if line.strip() else 999
                if line.strip() and current_indent <= theme_block_indent:
                    in_theme_block = False
                    fixed_lines.append(line)
                else:
                    if line.strip():
                        fixed_lines.append('    # ' + line.strip())
                    else:
                        fixed_lines.append(line)
                continue

            fixed_lines.append(line)

        with open(ui_file, 'w', encoding='utf-8') as f:
            f.write('\n'.join(fixed_lines))
    print("   ✅ Fixed")

print("\n🔧 Step 7: Updating llama module...")

LLAMA_CODE = f'''"""llama.cpp server"""
import json, torch, os, socket, subprocess, sys, threading, time, requests
from pathlib import Path
from modules import shared
from modules.logging_colors import logger

def get_llama_server_path():
    found = "{llama_server_path}"
    if os.path.exists(found): return found
    try:
        import llama_cpp
        for p in [Path(llama_cpp.__file__).parent/"server"/"llama-server", Path(sys.prefix)/"bin"/"llama-server"]:
            if p.exists(): return str(p)
        r = subprocess.run(['which','llama-server'], capture_output=True, text=True, timeout=5)
        if r.returncode == 0 and r.stdout.strip(): return r.stdout.strip()
    except: pass
    return "llama-server"

class LlamaServer:
    def __init__(self, model_path, server_path=None):
        self.model_path = model_path
        self.server_path = server_path or get_llama_server_path()
        logger.info(f"Llama-server: {{self.server_path}}")
        self.port = self._find_port()
        self.process = None
        self.session = requests.Session()
        self.vocabulary_size = None
        self.bos_token = "~~"
        self.last_prompt_token_count = 0
        self._start_server()

    def encode(self, text, add_bos_token=False, **kwargs):
        if self.bos_token and text.startswith(self.bos_token): add_bos_token = False
        r = self.session.post(f"http://127.0.0.1:{{self.port}}/tokenize", json={{"content":text,"add_special":add_bos_token}})
        return r.json().get("tokens",[])

    def decode(self, token_ids, **kwargs):
        r = self.session.post(f"http://127.0.0.1:{{self.port}}/detokenize", json={{"tokens":token_ids}})
        return r.json().get("content","")

    def prepare_payload(self, state):
        temp = state["temperature"]
        if state.get("dynamic_temperature"): temp = (state["dynatemp_low"]+state["dynatemp_high"])/2
        return {{"temperature":temp,"top_k":state.get("top_k",40),"top_p":state.get("top_p",0.95),
                "min_p":state.get("min_p",0.05),"repeat_penalty":state.get("repetition_penalty",1.1),"seed":state.get("seed",-1)}}

    def is_multimodal(self): return False

    def generate_with_streaming(self, prompt, state):
        payload = self.prepare_payload(state)
        tokens = self.encode(prompt, add_bos_token=state.get("add_bos_token",False))
        self.last_prompt_token_count = len(tokens)
        payload["prompt"] = tokens
        payload.update({{"n_predict":state.get('max_new_tokens',200),"stream":True,"cache_prompt":True}})
        r = self.session.post(f"http://127.0.0.1:{{self.port}}/completion", json=payload, stream=True)
        full = ""
        try:
            for line in r.iter_lines():
                if not line: continue
                try:
                    line = line.decode('utf-8')
                    if line.startswith('data: '): line = line[6:]
                    data = json.loads(line)
                    if data.get('content'): full += data['content']; yield full
                    if data.get('stop'): break
                except: continue
        finally: r.close()

    def generate(self, prompt, state):
        out = ""
        for out in self.generate_with_streaming(prompt, state): pass
        return out

    def _get_vocabulary_size(self):
        try:
            r = self.session.get(f"http://127.0.0.1:{{self.port}}/v1/models").json()
            if r.get("data") and r["data"]:
                meta = r["data"][0].get("meta",{{}})
                if "n_vocab" in meta: self.vocabulary_size = meta["n_vocab"]
        except: pass

    def _get_bos_token(self):
        try:
            r = self.session.get(f"http://127.0.0.1:{{self.port}}/props").json()
            if "bos_token" in r: self.bos_token = r["bos_token"]
        except: pass

    def _find_port(self):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.bind(('',0))
            return s.getsockname()[1]

    def _start_server(self):
        has_gpu = torch.cuda.is_available()
        gpu_layers = getattr(shared.args,'gpu_layers',0) if has_gpu else 0
        if has_gpu: logger.info(f"GPU: {{torch.cuda.get_device_name(0)}}")
        else: logger.info("CPU mode")
        cmd = [self.server_path,"--model",self.model_path,"--ctx-size",str(getattr(shared.args,'ctx_size',2048)),
               "--batch-size",str(getattr(shared.args,'batch_size',512)),"--port",str(self.port),"--no-webui"]
        if has_gpu and gpu_layers > 0: cmd += ["--gpu-layers",str(gpu_layers)]
        threads = getattr(shared.args,'threads',0)
        if threads > 0: cmd += ["--threads",str(threads)]
        logger.info(f"Starting llama.cpp on port {{self.port}}")
        try:
            self.process = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=subprocess.PIPE, bufsize=0)
        except FileNotFoundError:
            logger.error(f"llama-server not found: {{self.server_path}}")
            raise RuntimeError(f"llama-server not found: {{self.server_path}}")
        threading.Thread(target=self._log_stderr, daemon=True).start()
        health = f"http://127.0.0.1:{{self.port}}/health"
        for _ in range(60):
            if self.process.poll() is not None: raise RuntimeError(f"Server died: {{self.process.returncode}}")
            try:
                if self.session.get(health, timeout=1).status_code == 200: break
            except: pass
            time.sleep(1)
        self._get_vocabulary_size()
        self._get_bos_token()
        logger.info("Ready")

    def _log_stderr(self):
        try:
            for line in iter(self.process.stderr.readline, b''):
                print(line.decode('utf-8',errors='replace').strip(), file=sys.stderr)
        except: pass

    def stop(self):
        if self.process:
            self.process.terminate()
            try: self.process.wait(timeout=5)
            except: self.process.kill()

    def __del__(self): self.stop()
'''

(REPO_DIR / "modules" / "llama_cpp_server.py").write_text(LLAMA_CODE, encoding='utf-8')
print("   ✅ Fixed")

# ═══════════════════════════════════════════════════════════════════════════
# STEPS 8-10: DATA, MODEL, LINKS
# ═══════════════════════════════════════════════════════════════════════════

print("\n🔗 Step 8: User data...")
local_data = REPO_DIR / "user_data"
if local_data.exists() and not local_data.is_symlink():
    for item in local_data.rglob("*"):
        if item.is_file():
            rel = item.relative_to(local_data)
            dest = USER_DATA_DIR / rel
            dest.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(item, dest)
    shutil.rmtree(local_data)
if not local_data.exists():
    local_data.symlink_to(USER_DATA_DIR)
for sub in ["logs","logs/chat","logs/instruct","presets","characters"]:
    (USER_DATA_DIR / sub).mkdir(parents=True, exist_ok=True)
print("   ✅ Linked")

print("\n⬇️  Step 9: Model...")
if MODEL_FILE.exists():
    print(f"   ✅ Exists ({MODEL_FILE.stat().st_size/(1024**3):.2f} GB)")
else:
    print("   📥 Downloading...")
    MODEL_FILE.parent.mkdir(parents=True, exist_ok=True)
    run_cmd(["wget","-q","--show-progress","-O",str(MODEL_FILE),
            "https://huggingface.co/TheBloke/Llama-2-7B-GGUF/resolve/main/llama-2-7b.Q4_K_M.gguf"],
            quiet=False, timeout=900)

print("\n🔗 Step 10: Models link...")
repo_models = REPO_DIR / "models"
if not repo_models.is_symlink():
    if repo_models.exists():
        shutil.rmtree(repo_models)
    repo_models.symlink_to(MODELS_DIR)
print("   ✅ Linked")

print("\n🖥️  Step 11: Hardware...")
has_gpu = False
try:
    import torch
    has_gpu = torch.cuda.is_available()
    print(f"   {'✅ GPU: '+torch.cuda.get_device_name(0) if has_gpu else 'ℹ️  CPU'}")
except:
    print("   ℹ️  CPU")

hw = "GPU" if has_gpu else "CPU"

# ═══════════════════════════════════════════════════════════════════════════
# STEP 12: LAUNCH WITH AGGRESSIVE URL CAPTURE
# ═══════════════════════════════════════════════════════════════════════════

print("\n⚙️  Step 12: Launching with URL capture...")

def launch_server(attempt=1, max_attempts=3):
    """Launch server and capture URLs with retry logic"""

    print(f"\n{'='*70}")
    print(f"🚀 LAUNCH ATTEMPT {attempt}/{max_attempts}")
    print(f"{'='*70}")

    cmd = [sys.executable, "server.py"]
    if has_gpu:
        cmd.extend(["--gpu-layers", "35"])
    else:
        cmd.extend(["--cpu"])
    cmd.extend(["--threads","4","--listen","--listen-host","0.0.0.0","--share",
                "--model",str(MODEL_FILE),"--loader","llama.cpp"])

    print(f"Hardware: {hw} | Model: {MODEL_FILE.name}")
    print(f"{'='*70}\n")

    os.chdir(REPO_DIR)

    found_urls = {'local': None, 'public': None}
    urls_printed = [False]
    start_time = time.time()
    timeout = 180  # 3 minutes timeout

    try:
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                                  universal_newlines=True, bufsize=1)

        print("📡 Monitoring for Gradio URLs...\n")

        for line in process.stdout:
            # Print the line
            print(line, end='')

            # Extract URLs from this line
            line_urls = extract_urls(line)

            # Update our found URLs
            if 'local' in line_urls and not found_urls['local']:
                found_urls['local'] = line_urls['local']
                print(f"\n✅ LOCAL URL DETECTED: {found_urls['local']}\n")

            if 'public' in line_urls and not found_urls['public']:
                found_urls['public'] = line_urls['public']
                print(f"\n🌐 PUBLIC URL DETECTED: {found_urls['public']}\n")

            # If we have both URLs, print the box
            if found_urls['public'] and not urls_printed[0]:
                urls_printed[0] = True
                print_url_box(found_urls['local'], found_urls['public'], hw)

            # Check timeout
            if time.time() - start_time > timeout:
                print(f"\n⚠️  Timeout after {timeout}s")
                break

        # Keep process running if URLs found
        if urls_printed[0]:
            print("\n✅ Server running! URLs displayed above.")
            print("Press Ctrl+C to stop...\n")
            process.wait()
        else:
            print(f"\n⚠️  URLs not detected in attempt {attempt}")
            process.terminate()
            return found_urls

    except KeyboardInterrupt:
        print("\n⏹️  Stopped by user")
        if found_urls['public']:
            print(f"📝 Public URL was: {found_urls['public']}")

    except Exception as e:
        print(f"\n❌ Error in attempt {attempt}: {e}")
        return None

    return found_urls

# Try to launch
result = None
for attempt in range(1, 4):  # Try up to 3 times
    result = launch_server(attempt, max_attempts=3)

    if result and result.get('public'):
        # Success! We got the public URL
        break
    elif attempt < 3:
        print(f"\n🔄 Retrying in 5 seconds...")
        time.sleep(5)
    else:
        print(f"\n❌ Failed to get public URL after {attempt} attempts")
        print("Check the output above for errors")

print("\n✅ Done")